# 1-D Gridworld

This notebook contains an implementation of a Policy Iteration algorithm to estimate and solve a simple Gridworld problem. The policy to be optimized is initially an equiprobable (thus, stochastic) policy, but then converges to a deterministic policy (s.t. $\pi(a|s)=1$ for one $a$ and $\pi(a|s)=0$ otherwise. The policy iteration utilizes an iterative policy evaluation. An $\epsilon$-soft policy is not used.

Note that $p(s',r|s,a)=1$.

In [1]:
using Random

In [2]:
gamma = 1

1

In [3]:
mutable struct Gridworld
    rewards::Array{Int32}
end

num_states(g::Gridworld) = length(g.rewards)

is_terminal_state(g::Gridworld, state::Int32) = state == 1 || state == num_states(g)

"""
Performs an action on the gridworld on the given state and returns the recieved reward and the succeeding state.

# Arguments
- `action`: true for right and false for left
"""
function act(g::Gridworld, state::Int64, action::Bool)
    reward = 0
    if action == 0
        new_state = max(1, state - 1)
    elseif action == 1
        new_state = min(num_states(g), state + 1)
    end
    if new_state != state
        reward = g.rewards[new_state]
    end
    reward, new_state
end

act (generic function with 1 method)

In [7]:
"""
Solves the gridworld problem using a policy iteration algorithm.

# Arguments
- `g`: The gridworld to be solved
- `gamma`: Reward discounting hyperparameter
- `tol`: Small value used to decide whether the policy evaluation converged

# Returns
- value function as array of length `num_states(g)`
- optimal policy as a boolean array of length `num_states(g)`
"""
function policy_iteration(g::Gridworld, gamma, tol=1e-2)
    delta = tol
    policy_stable = false
    values = randn(num_states(g))  # Fill value table arbitrarily, but terminal states must be 0
    values[1] = 0
    values[num_states(g)] = 0
    policy = fill(.5, num_states(g))  # P(right), P(left) = 1 - P(right)
    
    while !policy_stable
        # Policy Evaluation
        while delta >= tol
            delta = 0
            for state in 1:num_states(g)
                old_value = values[state]
                # Only two actions possible, thus it is efficient enough to compute both
                right_reward, right_new_state = act(g, state, true)
                values[state] = policy[state] * (right_reward + gamma * values[right_new_state])
                left_reward, left_new_state = act(g, state, false)
                values[state] += (1 - policy[state]) * (left_reward + gamma * values[left_new_state])
                delta = max(delta, abs(old_value - values[state]))
            end
        end

        # Policy improvement
        policy_stable = true
        for state in 1:num_states(g)
            old_action = policy[state] > 0.5  # old action is right if P(right) > 0.5 and left otherwise
            right_reward, right_new_state = act(g, state, true)
            left_reward, left_new_state = act(g, state, false)
            # if right action is more optimal, set P(right) = 1, otherwise P(right) = 0 => deterministic policy
            if right_reward + gamma * values[right_new_state] > left_reward + gamma * values[left_new_state]
                policy[state] = 1
            else
                policy[state] = 0
            end
            if old_action != policy[state]
                policy_stable = false
            end
        end
    end
    values, policy
end

policy_iteration

In [5]:
gridworld = Gridworld([10 0 0 0 0 0 0 0 0 -5])

Gridworld(Int32[10 0 … 0 -5])

In [8]:
policy_iteration(gridworld, gamma)

([15.247462870058783, 20.325074323458395, 15.396734688852531, 10.461472881762514, 5.518434468014517, 0.5668925671546345, -4.3937432903252756, -9.36392045261873, -14.343938650542206, -14.333947749503945], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])